In [14]:
%%html
<div>
<font style="font-size: xx-large; font-weight: bold">Using ECDH and symmetric cryptography to build digital signatures</font>
</div>
<div>By Ronald Landheer-Cieslak</div>
<div style="text-align: right">
<div><a target="_blank" href="index.pdf">Download PDF version</a></div>
<div><a target="_blank" href="https://github.com/blytkerchan/ecdh-signatures.git">GitHub</a></div>
</div>

# Introduction

Digital signatures are used in every-day life to determine the authenticity of any number of messages of any type, from websites and APIs to E-mail and other messages. Whenever someone connects to a a website with HTTPS, digital signatures are involved in the authentication of that website and the public key infrastructure (PKI) used to establish trust with the server. These digital signatures are based on a set of algorithms collectively called "digital signature algorithms" that include RSA and ECDSA as well as some of the newer quantum-resistant algorithms such as "Dilithium" and "Sphincs+".

Diffie-Hellman (DH) key exchange algorithms do not normally serve to sign existing messages, but rather serve to generate a shared symmetric key. That shared symmetric key can then be used for any purpose, including authenticated encryption with additional data (AEAD) using algorithms such as AES-GCM, or authentication using symmetric keys employing hash-based message authentication codes (HMAC).

Key encapsulation mechanisms (KEM) are another way of sharing symmetric keys, in this case not generating them using a private and a public key, but randomly generating and then sharing them with a specific recipient using the recipient's public key. Contrary to a Diffie-Hellman-type algorithm, a KEM does not require the sender's private key, but only requires the recipient's public key to encrypt the key. The post-quantum algorithm "Kyper" is a key encapsulation mechanism.

Until recently, one of the most promising post-quantum asymmetric cryptography algorithms was an algorithm from the family of DH algorithms, and DSA and KEM algorithms seemed less promising both in their efficiency and their security. That was until July 2022, when an [attack](https://eprint.iacr.org/2022/1026) on Supersingular Isogeny Diffie-Hellman (SIDH) was found that can be performed on a classical computer and thoroughly breaks the security of SIDH. There is still hope, however, with lattice-based algorithms, and with algorithms based in learning-with-errors.

Diffie-Hellman-type algorithms are arguably the most valuable assymetric algorithms known to man: they are typically used for generating shared secret keys, but they *could* be used for signatures as well. In this paper, I will argue for this assertion by presenting a protocol for signing and verifying using a Diffie-Hellman-type construct in conjunction with quantum-safe symmetric cryptography. Grover's algorithm notwithstanding, symmetric cryptography *is safe* as soon as one doubles the key sizes vs. before quantum computers were a thing.

I am obviously not presenting a post-quantum Diffie-Hellman-type algorithm: I'm not that good at math.

# The problem with digital signature algorithms

A digital signature algorithm (DSA) consists of three functions: a function to create a keypair, a function to sign, and a function to verify. The function to generate a keypair generates two keys, which we will call $sk$ for the private key and $pk$ for the public key $$sk, pk = G$$ The signing function $S$ is defined as $$sig = S~sk~m$$ where $sig$ is the signature and $m$ is the message. The verification function $V$ is defined as $$r = V~pk~m~sig$$ where r is a boolean result.

Digital signature algorithms are not suitable to share a key: if Alice wants to send a message $m_{a\rightarrow b}$ with Bob, she can use a DSA to sign that message, but anyone with her public key $pk_a$ can verify that signature. She cannot target the signature to be readable only by Bob and she cannot use this to encrypt her message to Bob.

# The problem with key encapsulation mechanisms

A key encapsulation mechanism also consists of three functions: a function to create a keypair, a function to encrypt, and a function to decrypt. The function to create the keypair is essentially the same: $$sk, pk = G$$ The function to encrypt generally only takes the public key of the target recipient, $pk_b$, as parameter and generates both the symmetric secret $s$ and the ciphertext $c$, so this function is defined as $$s, c = E~pk_b$$ The function to decrypt takes the ciphertext and the private key as parameters and reproduces the shared secret: $$s = D~sk_b~c$$

The way this would typically be used in an on-line key exchange would be for the server to generate a keypair and send it to the client, the for the client to encrypt the shared key and send the ciphertext back to the server, at which time the server can decrypt it. Obviously, this means the server needs some sort of digital signature algorithm to sign the public key it sends to the client, and cannot authenticate the client unless it uses its own DSA as well, or an application-level authentication is also used.

# The comparative strength of a Diffie-Hellman-type key exchange

This paper presents a protocol in which Diffie-Hellman can be used for digital signatures. While the intent is not to present a protocol for Diffie-Hellman to be used for key encapsulation, such a protocol would be trivial. Diffie-Hellman consists of two functions: a keypair generation function $$sk, pk = G$$ and a shared secret generation that takes the local private key and the remote public key as parameters: $$s = H~sk_a~pk_b = H~sk_b~pk_a$$

While this paper presents a protocol for DSA using DH, implementing KEM using DH is trivial: keypair generation remains the same $$sk, pk = G$$

The encrypt function generates an ephemeral keypair, performs the DH using its private key, and returns the shared secret and the ephemeral public key, the latter as "ciphertext" to be shared with the other party:
$$
s, c = E~pk_b :\\
sk_a, pk_a = G\\
s = H~sk_a~pk_b\\
c = pk_a
$$
The decrypt function takes the local private key and the "ciphertext", which is an ephemeral public key, and produces the same shared secret:
$$
s = D~sk_b~c :\\
D = H
$$

Hence, with the protocol laid out below, DH-type algorithms can implement both DSA and (trivially, as shown) KEM. 

# The protocol: signing

Given a function $G$ that generates a public and private key suitable for use in the following functions

Given a function $D$ taking a private key $sk_a$ and a public key $pk_b$, and outputting a shared secret $k$.

Given a function $P$ that will transform a single large integer into a private key suitable for use with the function $D$.

Given a function $U$ that computes a public key from a given private key.

Given a secure hash function $H$.

Given a secure HMAC function $M$.

Given an HKDF function using the underlying HMAC $M$, $K_M$

Given a message from Alice to Bob $m_{a\rightarrow b}$.

Given Alice's keypair $sk_a, pk_a = G$.

Compute $h_1 = H~m_{a\rightarrow b}$.

Compute $h_2 = H~h_1 | m_{a\rightarrow b}$ where $|$ is a concatenation operator.

Compute $sk' = P~h_1'$.

Compute $pk' = U~sk'$

Compute $k = D~sk_a~pk'$

Compute $s = K_M~h_2~k$

Compute $m' = M~s~m_{a\rightarrow b}$

Send $\langle m_{a\rightarrow b},m'\rangle$ to Bob.

## The protocol: verifying

Given the same functions $D$, $P$, $H$, $M$ and $K_M$ ($U$ is not needed in this context).

Given Alice's public key $pk_a$.

Given Alice's message $\langle m_{a\rightarrow b},m'\rangle$.

Compute $h_1' = H~m_{a\rightarrow b}$.

Compute $h_2' = H~h_1' | m_{a\rightarrow b}$ where $|$ is a concatenation operator.

Compute $sk' = P~h_1$.

Compute $k' = D~sk'~pk_a$

Compute $s' = K_M~h_2'~k'$

Compute $m'' = M~s'~m_{a\rightarrow b}$

Verify that $m' == m''$. If so, the message is verified as having been signed with $sk_a$.

# Implementing with ECDH

In the context of this paper, we will use Elliptic Curve Diffie-Hellman as an example of a Diffie-Hellman-type key negotiation scheme, but the only requirements on the scheme, represented here by the function $D$, are that:
1. the function $D$ takes a private key and a public key as parameters and
2. a private key suitable as a parameter to the function $D$ can be deterministically generated from a single large integer value

We will be doing this in Python:

In [ ]:
import tinyec.ec as ec
import tinyec.registry as reg
import os
import lipsum
import hashlib
import hmac
from hkdf import *


When using ECDH for the function $D$, we can choose an elliptic curve $C = \langle p, a, b, g, n, h \rangle$ or $C = \langle m, f, a, b, g, n, h \rangle$ where the former option is a prime curve and the second option is a binary curve; $p$ or $m, f$ determine the size of the curve $a, b$ are the defining parameters of the elliptic curve, $g$ is the generator, $n$ is the order of the point at infinity, and $h$ is the cofactor. These parameters are all public.

In [ ]:
curve = reg.get_curve('secp521r1')

We do, of course, need the function to generate a keypair, $G$, as well.

> Given a function $G$ that generates a public and private key suitable for use in the following functions

In [ ]:
def G():
    sk = int.from_bytes(os.urandom(66), 'big') % (2 ** 521)
    pk = sk * curve.g
    return sk, pk

## Alice: signing

> Given a function $D$ taking a private key $sk_a$ and a public key $pk_b$, and outputting a shared secret $k$.

In [ ]:
def D(sk, pk):
    k = sk * pk
    return (k.x * 16 + k.y % 2).to_bytes(525+7//8, 'big')

> Given a function $P$ that will transform a single large integer into a private key suitable for use with the function $D$.

In [ ]:
def P(h):
    '''$$P_C(h) = h \mod n$$'''
    return int.from_bytes(h, 'big') % curve.field.n

> Given a function $U$ that computes a public key from a given private key.

In [ ]:
def U(pk):
    return pk * curve.g

> Given a secure hash function $H$.

In [ ]:
def H(data):
    sha = hashlib.sha256(data)
    return sha.digest()

> Given a secure HMAC function $M$.

In [ ]:
def M(key, data):
    if isinstance(data, str):
        data = data.encode('utf-8')
    h = hmac.new(key, msg=data, digestmod='sha256')
    return h.digest()

> Given an HKDF function using the underlying HMAC $M$, $K_M$

In [ ]:
def K(salt, ikm, length=32):
    prk = hkdf_extract(salt, ikm, hash=hashlib.sha256)
    return hkdf_expand(prk, length=length)

> Given a message $m_{A\rightarrow B}$

In [ ]:
m = lipsum.generate_paragraphs(3)

> Given Alice's keypair $sk_a, pk_a = G$.

In [ ]:
sk_a, pk_a = G()

> compute $h_1 = H(m_{A\rightarrow B})$

In [ ]:
h_1 = H(m.encode('utf-8'))

> Compute $h_2 = H~h_1 | m_{a\rightarrow b}$ where $|$ is a concatenation operator.

In [ ]:
h_2 = H(b''.join([h_1, m.encode('utf-8')]))

> Compute $sk' = P~h_1$

In [ ]:
sk_prime = P(h_1)

> Compute $pk' = U~sk'$

In [ ]:
pk_prime = U(sk_prime)

> Compute $k = D~sk_a~pk'$

In [ ]:
k = D(sk_a, pk_prime)

> Compute $s = K_M~h_2~k$

In [ ]:
s = K(h_2, k)

> Compute $m' = M~s~m_{a\rightarrow b}$

In [ ]:
m_prime = M(s, m)

> Send $\langle m_{a\rightarrow b},m'\rangle$ to Bob.

In [ ]:
to_bob = m, m_prime

## Bob: verifying

> Given the same functions $D$, $P$, $H$, $M$ and $K_M$ ($U$ is not needed in this context).

In [ ]:
# inherited from cells above

> Given Alice's public key $pk_a$.

In [ ]:
# inherited from cells above

> Given Alice's message $\langle m_{a\rightarrow b},m'\rangle$.

In [ ]:
m, m_prime = to_bob

> Compute $h_1' = H~m_{a\rightarrow b}$.

In [ ]:
h_1_prime = H(m.encode('utf-8'))

> Compute $h_2' = H~h_1' | m_{a\rightarrow b}$ where $|$ is a concatenation operator.

In [ ]:
h_2_prime = H(b''.join([h_1_prime, m.encode('utf-8')]))

> Compute $sk' = P~h_1'$.

In [ ]:
sk_prime = P(h_1_prime)

> Compute $k' = D~sk'~pk_a$

In [ ]:
k_prime = D(sk_prime, pk_a)

> Compute $s' = K_M~h_2'~k'$

In [ ]:
s_prime = K(h_2_prime, k_prime)

> Compute $m'' = M~s'~m_{a\rightarrow b}$

In [ ]:
m_double_prime = M(s_prime, m)

> Verify that $m' == m''$. If so, the message is verified as having been signed with $sk_a$.

In [ ]:
m_prime == m_double_prime